# Deep leanring project

## 1 Data loading and argumenting

In [0]:
##### before running it, make sure you don't have lots of big files in your google drive
##### otherwise it's going to take too long to finish running it before giving the TIMEOUT error
##### also save the train_controls, train_patients, val_controls, val_patients to your drive and
##### create a "train" folder with train_controls, train_patients in it, and 
##### a "val" folder with val_controls, val_patients in it.
##### change the train_dir and val_dir in the next cell to the dir of your train and val folder

from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


## 1.1 Seeds and predefined stuffs

In [0]:
from numpy.random import seed
seed(137)
from tensorflow import set_random_seed
set_random_seed(191)

# Dir (Comment out others when you run the code)
# e.g. /gdrive/My Drive/deep_learning/new_dataset/test/controls/View2098.jpg
#          |                                             |

# Kavi's

# Daniel's

# Chelsea's Probs
train_dir = "/gdrive/My Drive/deep_learning/EGD"


img_width = 224
img_height = 224
batch_size = 5
channels = 3

## 1.2 Data loading

In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)             
# val_datagen = ImageDataGenerator(rescale=1./255)              

train_generator = train_datagen.flow_from_directory(
        train_dir, 
        target_size=(img_height, img_width),
        batch_size=batch_size,
        shuffle = True,
        class_mode='binary')   

#Keras takes care of generating labels if the directory structure matches above!
label_mapT = train_generator.class_indices
print(label_mapT)

for data_batch, labels_batch in train_generator:
    print ('data batch shape:', data_batch.shape)
    #print(data_batch)
    print('labels batch shape:', labels_batch.shape)
    #print(labels_batch)
    break
    
nb_train_samples = len(train_generator.filenames)

Using TensorFlow backend.


Found 102 images belonging to 2 classes.
{'controls': 0, 'patients': 1}


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:2274: DecompressionBombWarning: Image size (171407884 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


data batch shape: (5, 224, 224, 3)
labels batch shape: (5,)


In [0]:
import numpy as np

def stacking_samples(dataset_type, sample_count):
    features = np.zeros(shape=(sample_count, img_height, img_width, channels))
    labels = np.zeros(shape=(sample_count))
    i = 0
    if dataset_type == "train":
        for inputs_batch, labels_batch in train_generator:
#             features_batch = conv_base.predict(inputs_batch)
            features[i * batch_size : (i + 1) * batch_size] = inputs_batch
            labels[i * batch_size : (i + 1) * batch_size] = np.argmax(labels_batch)
            i += 1
            print(i * batch_size)
            if i * batch_size >= sample_count:
                break   
#     elif dataset_type == "valid":
#         for inputs_batch, labels_batch in valid_generator:
# #             features_batch = conv_base.predict(inputs_batch)
#             features[i * batch_size : (i + 1) * batch_size] = inputs_batch
#             labels[i * batch_size : (i + 1) * batch_size] = np.argmax(labels_batch)
#             i += 1
#             if i * batch_size >= sample_count:
#                 break
#     else:
#         for inputs_batch, labels_batch in test_generator:
# #             features_batch = conv_base.predict(inputs_batch)
#             features[i * batch_size : (i + 1) * batch_size] = inputs_batch
#             labels[i * batch_size : (i + 1) * batch_size] = np.argmax(labels_batch)
#             i += 1
#             if i * batch_size >= sample_count:
#                 break
    return features, labels


train_features, train_labels = stacking_samples("train", nb_train_samples)


print(train_features.shape, train_labels.shape)

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:2274: DecompressionBombWarning: Image size (171407884 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
(102, 224, 224, 3) (102,)


## 2 Model training

In [0]:
import numpy as np 
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Softmax, Flatten, Dense, BatchNormalization, GlobalAveragePooling2D
from keras.layers import Lambda
from keras.metrics import categorical_accuracy
from keras import regularizers
import tensorflow as tf
from keras.models import Sequential
from keras import optimizers

from keras.callbacks import TensorBoard

#from keras.layers import Input, Dense
from keras.models import Model

input_shape = (img_height, img_width, channels)


def getModel():
  model = Sequential()

  model.add(Conv2D(32,(11, 11), input_shape=input_shape))#, kernel_regularizer=regularizers.l1(0.01))) 
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(BatchNormalization())

  model.add(Conv2D(32,(3, 3), input_shape=input_shape))#, kernel_regularizer=regularizers.l1(0.01))) 
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(BatchNormalization())

  model.add(Conv2D(32,(3, 3), input_shape=input_shape))#, kernel_regularizer=regularizers.l1(0.01))) 
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(BatchNormalization())


  model.add(Flatten())
  model.add(Dense(128))
  model.add(Activation('relu'))
  model.add(Dropout(0.5)) 
  model.add(Dense(1))
  model.add(Activation('sigmoid'))
  
  opt = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.1)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  
  return model

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 214, 214, 32)      11648     
_________________________________________________________________
activation_6 (Activation)    (None, 214, 214, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 107, 107, 32)      0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 107, 107, 32)      128       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 105, 105, 32)      9248      
_________________________________________________________________
activation_7 (Activation)    (None, 105, 105, 32)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 52, 52, 32)        0         
__________

In [0]:
from keras import optimizers

opt = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.1)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

## Early Stopping

In [0]:
# For early stopping
import keras
from keras.callbacks import TensorBoard, Callback, EarlyStopping

class MetricsCheckpoint(Callback):
    """Callback that saves metrics after each epoch"""
    def __init__(self, savepath):
        super(MetricsCheckpoint, self).__init__()
        self.savepath = savepath
        self.history = {}
    def on_epoch_end(self, epoch, logs=None):
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        np.save(self.savepath, self.history)
        
callbacks_list = [EarlyStopping(monitor='val_acc', patience=20, verbose=1)]

## 10-fold

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn import metrics

features = train_features
labels = train_labels

kf = KFold(n_splits=10, shuffle=True)

epochs = 20

accs = []
for train_index, val_index in kf.split(features):
  x_train = features[train_index]
  y_train = labels[train_index]
  x_val = features[val_index]
  y_val = labels[val_index]
  
  model = getModel()

  history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_val, y_val))
  accs.append(history.history['val_acc'][-1])

print(accs)

Train on 91 samples, validate on 11 samples
Epoch 1/20
91/91 [==============================] - 9s 104ms/step - loss: 2.4585 - acc: 0.6484 - val_loss: 3.0460 - val_acc: 0.7273
Epoch 2/20
91/91 [==============================] - 1s 8ms/step - loss: 1.2230 - acc: 0.7692 - val_loss: 2.8968 - val_acc: 0.6364
Epoch 3/20
91/91 [==============================] - 1s 8ms/step - loss: 0.4275 - acc: 0.8681 - val_loss: 2.5990 - val_acc: 0.6364
Epoch 4/20
91/91 [==============================] - 1s 8ms/step - loss: 0.5070 - acc: 0.8681 - val_loss: 2.8275 - val_acc: 0.7273
Epoch 5/20
91/91 [==============================] - 1s 8ms/step - loss: 0.4353 - acc: 0.8901 - val_loss: 3.2180 - val_acc: 0.5455
Epoch 6/20
91/91 [==============================] - 1s 8ms/step - loss: 0.1251 - acc: 0.9670 - val_loss: 3.1026 - val_acc: 0.5455
Epoch 7/20
91/91 [==============================] - 1s 8ms/step - loss: 0.1609 - acc: 0.9560 - val_loss: 3.4208 - val_acc: 0.5455
Epoch 8/20
91/91 [==========================

In [0]:
ave = np.mean(accs)
print("final accuracy", ave)

final accuracy 0.7363636469976468
